Add the project root to Python's import path so local modules (src/...) can be imported easily.

In [ ]:
import sys
from pathlib import Path

project_path = Path.cwd().parent.parent

sys.path.append(str(project_path.resolve()))

Get the API Keys

In [ ]:
import os

HF_TOKEN = os.getenv("HF_TOKEN", "False")
WANDB_API_KEY = os.getenv("WANDB_API_KEY", "False")

Import dataset helpers, evaluation utilities, Hugging Face Transformers and PEFT components used for training and tokenization.

In [ ]:
from huggingface_hub import login

login(token=HF_TOKEN)

In [ ]:
import wandb

In [ ]:
from src.dataset.load_data_soda import SODADataLoader
from src.utils.story_eval import get_compute_metrics_function_for_stories
from config.dialogue_special_tokens import update_sep_token, DIALOGUE_END_TOKEN
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from peft import (
    get_peft_model,
    LoraConfig,
    TaskType
)

Enable tokenizer parallelism via environment variable. This can speed up tokenization on multi-core machines.

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"

Create the SODA dataset loader with simple filtering options and retrieve the dataset object.

In [ ]:
soda_dataset_obj = SODADataLoader(
    percent_of_all_splits=10,
    min_story_length=20,
    max_story_length=250,
    unroll_dialogue_and_speakers=True
)
soda_dataset_obj.duplicate_eod_examples(min_dupl=3, max_dupl=5)
soda_ds = soda_dataset_obj.dataset

Load Tokenizer and Model from HuggingFace

In [ ]:
model_name = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Update tokenizer with dialogue end token

In [ ]:
update_sep_token(tokenizer.sep_token)

tokenizer.add_special_tokens({
    'additional_special_tokens': [
        DIALOGUE_END_TOKEN
    ]
})

Resize model embeddings to match new tokenizer size

In [ ]:
base_model.resize_token_embeddings(len(tokenizer))

Enable gradient checkpointing compatibility

In [ ]:
base_model.enable_input_require_grads()

Set LoRA finetuning config values.

In [ ]:
lora_config = LoraConfig(
    r=16,  # Rank of the update matrices. Lower r = fewer parameters.
    lora_alpha=32,  # Alpha scaling factor.
    target_modules=["q_proj", "v_proj"],  # Apply LoRA to query and value projections in attention
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM  # Specify the task type for seq2seq models
)

Create the PEFT model using the base model and LoRA configuration

In [ ]:
model = get_peft_model(base_model, lora_config)

In [ ]:
def get_model_parameter_counts(model):
    """
    Calculates and returns the total parameters, trainable parameters, 
    and the percentage of trainable parameters.
    """
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel()
                           for p in model.parameters() if p.requires_grad)
    trainable_percentage = (trainable_params / total_params) * 100

    return {
        "total_params": total_params,
        "trainable_params": trainable_params,
        "trainable_percentage": trainable_percentage
    }

Define a preprocessing function to tokenize inputs (narrative) and targets (dialogue). Then map it over the dataset.

In [ ]:
max_source_length = 900
max_target_length = 128


def preprocess(example):
    prefix = ""  # Some models benefit from a task-specific prefix

    # Add prefix to every input
    inputs_nar = [prefix + doc for doc in example["narrative"]]

    inputs = tokenizer(
        inputs_nar,
        max_length=max_source_length,
        truncation=True,
    )
    targets = tokenizer(
        text_target=example["dialogue"],
        max_length=max_target_length,
        truncation=True,
    )
    inputs["labels"] = targets["input_ids"]
    return inputs


tokenized_datasets = soda_ds.map(
    preprocess,
    batched=True,
    remove_columns=soda_ds["train"].column_names,
)

WANDB and HF repo settings

In [ ]:
wandb_project = os.getenv("WANDB_PROJECT", "story2dialogue-soda-bart-lora")
hf_repo_id = os.getenv("HF_REPO_ID", False)

Initialize Weights & Biases ([wandb](https://wandb.ai/)) logging

In [ ]:
wandb.init(
    project=wandb_project,
    config={
        "model_name": model_name,
        "dataset": "SODA",
        "max_source_length": max_source_length,
        "max_target_length": max_target_length,
    }
    | soda_dataset_obj.get_dataset_info(include_word_counts=True)
    | lora_config.__dict__
    | get_model_parameter_counts(model)
)

Create a data collator to batch examples correctly for seq2seq training.

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Set training arguments for the Hugging Face Trainer (epochs, batch size, save/eval strategies, etc.).

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./bart-soda-lora-finetuned-local",
    run_name="story2dialogue-SODA-BERT-LoRA-local",
    label_names=["labels"],
    
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,

    warmup_steps=500,

    learning_rate=1e-4,
    adam_epsilon=1e-8,
    label_smoothing_factor=0.0,
    weight_decay=0.01,
    
    fp16=True,
    seed=42,
    
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="rouge2",
    greater_is_better=True,
    predict_with_generate=True,
    # generation_num_beams=4,
    dataloader_num_workers=2,
    
    logging_strategy="steps",
    logging_steps=250,
    report_to="wandb",
    push_to_hub=True if hf_repo_id else False,
    hub_model_id=hf_repo_id,
    hub_strategy="every_save",
    
)

Prepare the function used to compute evaluation metrics during training and evaluation.

In [ ]:
compute_metrics_eval = get_compute_metrics_function_for_stories(tokenizer)

Create the Seq2SeqTrainer by wiring model, tokenizer, datasets, data collator and metric function together.

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics_eval
)

Start training the model. This will run for the number of epochs set in the training arguments.

In [ ]:
train_result = trainer.train()

Save the trained model and store training metrics and trainer state to disk.

In [ ]:
trainer.save_model("./bart-soda-lora-finetuned-local-final")
metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

Upload fine-tuned model to HuggingFace

In [ ]:
trainer.push_to_hub()

Run predictions on the test split to get model outputs for evaluation.

In [ ]:
raw_predictions = trainer.predict(tokenized_datasets["test"])

In [ ]:
cal_metrices_events_comp_fn = get_compute_metrics_function_for_stories(
    tokenizer,
    metrics_prefix="test/",
    save_preds=True,
    save_preds_filename=wandb.run.name+"_preds.csv",
    include_bertscore=True
)
metrics = cal_metrices_events_comp_fn(raw_predictions)

In [ ]:
wandb.finish()